<a href="https://colab.research.google.com/github/yutawatabe/pyCGE/blob/main/EK_Fullsolution_ENG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equilibrium Calculation

We simulate random parameters and calculate the equilibrium.

There are five steps
1. Set parameters
2. Calcualte an excess labor demand from the parmaters and wages.
3. Update wages from the excess labor demand.
4. Make step 2 and three to a function, and loop them over. Confirm that the excess labor demand converge to zero.
5. Confirm that there is no excess labor demand and output wages and trade flows.

In [ ]:
# import numpy and math
import numpy as np # Matrix calculation library
from math import gamma # For the price index calculation

# Set parameters

In [ ]:
N = 3 # Number of countries
theta = 4 # Trade elasticity parameter
sigma = 3 # Elasticity of substitution paramter

In [ ]:
# np.array is a function to create vectors.
#  Here we only deal with vectors (one-dimensional) and matrices (two-dimensional)
#  but of course it can also create multi-dimensional arrays.
T = np.array([1., 1., 1.]) # Technology parameter
# By using print, the contents of T are displayed.
# T.shape is used to get the shape of T (the shape of the vector).
print(T)
print(T.shape)

[1. 1. 1.]
(3,)


In [ ]:
L_S = np.array([1,1.5,1.5]) # Population
print(L_S)
print(L_S.shape)

[1.  1.5 1.5]
(3,)


Set $\boldsymbol{d}$. First set all the elements of $\boldsymbol{d}$ as 1.5, and then use for loop to make the domestic trade costs to be 1.

In [ ]:
d = np.ones((N,N)) * 1.5 # Create trade costs matrix
print(d)
print(d.shape)
# A for loop in Python can be written as follows.
# To be more precise, here we have a variable called OR, which is a loop that goes up from 0 to N-1 in order.
# By setting d[OR,OR] = 1, d[0,0] = 1, d[1,1] = 1, we set the domestic trade cost to 1 in each country.

for OR in np.ndindex((N)):
  d[OR,OR] = 1 #Domestic trade cost is 1

print(d)

[[1.5 1.5 1.5]
 [1.5 1.5 1.5]
 [1.5 1.5 1.5]]
(3, 3)
[[1.  1.5 1.5]
 [1.5 1.  1.5]
 [1.5 1.5 1. ]]


# Calculate excess labor demand

We first assume all the wages are equal and see if the labor market (good markets) clear. As for candiddate wages, we think of a following:
$$ w_i = 0.25. $$


In [ ]:
w = np.ones((N)) / 4
print(w)

[0.25 0.25 0.25]


First, let's calculate the total income (consumption). Let $X_n$ be the total consumption. In this economy, labor is the only factor of production, so $$Y_n = X_n = w_n L_n$$ where:

$Y_n$ is the total income, $X_n$ is the total consumption, $w_n$ is the nominal wage rate, $L_n$ is the labor input

In other words, in an economy where labor is the sole factor of production, the total income equals the total consumption, which is calculated by multiplying the nominal wage rate by the amount of labor input.

In [ ]:
# w * L will be an element-wise vector calculation (it's not an innter product).
Xn = w * L_S
print(Xn)

[0.25  0.375 0.375]


Before calculating the import share, I will explain about the two-dimensional loop. We can loop over multiple dimensional variables, and we will utilize it to the calculation of import share.


In [ ]:
# One-dimensional loop can be written as follows.
for OR in np.ndindex((N)):
  print(OR)

# Two-dimesional loop can be written as follows.
# We use OR and DE as looping variables.
# np.ndindex(N,N) can be seen as looping the elements (indices) of N×N matrix.
# Specifically,the variables moves like OR = 0, DE = 0 to OR = 0, DE = 1.
for OR,DE in np.ndindex((N,N)):
  print([OR,DE])

(0,)
(1,)
(2,)
[0, 0]
[0, 1]
[0, 2]
[1, 0]
[1, 1]
[1, 2]
[2, 0]
[2, 1]
[2, 2]


We calculate the import share. The formula is as follows:
$$ \pi_{in} = \frac{ T_{i} (w_{i} d_{in})^{-\theta} }{\sum_{k=1} T_{k} (w_{k} d_{kn})^{-\theta} } = \frac{X_{in}}{X_i} ,$$
where $X_{in}$ is export value from country i to country n. As for coding, we write as follows:
$$ \pi_{in} = \frac{\pi_{in,num}}{\Phi_{n}}, $$
where
$ \pi_{in,num} = T_{i} (w_{i} d_{in})^{-\theta} $ and $ \Phi_{n} = \sum_{k=1}^N T_{k} (w_{k} d_{kn})^{-\theta}. $



In [ ]:
# Create an initial matrix for the import share (the share of country i's goods in the total consumption of n countries).
# If this box doesn't exist, an error will occur when looping because there will be no box to fill.
pi = np.zeros((N, N))
pi_num = np.zeros((N, N))  # Numerator part of the import share
Phi = np.zeros((N))  # Denominator part of the import share

for OR, DE in np.ndindex((N, N)):
    # Use multiplicands in the calculation of pi_num. For example, **(−theta) means raised to the power of −theta.
    pi_num[OR, DE] = T[OR] * (w[OR] * d[OR, DE]) ** (-theta)

    # Add this pi_num[OR, DE] to create Phi[DE].
    # By repeating this recursion, Phi[DE] is the sum of pi_num[OR, DE] along the first dimension.
    Phi[DE] = Phi[DE] + pi_num[OR, DE]

# Divide the numerator by the denominator to obtain the import share.
for OR, DE in np.ndindex((N, N)):
    pi[OR, DE] = pi_num[OR, DE] / Phi[DE]

print(pi)

[[0.71681416 0.14159292 0.14159292]
 [0.14159292 0.71681416 0.14159292]
 [0.14159292 0.14159292 0.71681416]]


By the definition, $\sum_{i=1}^N \pi_{in} = 1$ should hold. Also, since the (tentative) wages are identical and the country's technology and trade costs are symmetric, $\boldsymbol{\pi}$ should be symmetric. We confirm this above.
From here, we calculate the price index (to be used later for welfare evaluation):
$$
P_n = \Gamma \left( \frac{\theta + \sigma -1}{\theta} \right) \Phi_n^{-1/\theta}
$$

In [ ]:
# gamma() is a gamma function.
P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)
print(P)

[0.24435385 0.24435385 0.24435385]


Export from country i to country n is an import share times total absorption of country n.
$$X_{ni} = \pi_{ni} X_n $$
The export value matrix does not sum to one and is not symmetric because the economic sizes differ across countries.

In [ ]:
X = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  X[OR,DE] = pi[OR,DE] * Xn[DE]

print(X)

[[0.17920354 0.05309735 0.05309735]
 [0.03539823 0.26880531 0.05309735]
 [0.03539823 0.05309735 0.26880531]]


Total labor demand is total sales divided by wages:

$$ L_{i,D} = \frac{ \sum_{n=1}^N X_{ni}}{w_i} $$


In [ ]:
L_D = np.zeros((N))
for OR,DE in np.ndindex((N,N)):
  L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]

print(L_D)

[1.14159292 1.42920354 1.42920354]


**Confirm labor makret equilibrium**

Labor supply is fixed $(\boldsymbol{L} = \boldsymbol{L}_S)$. We derive the excess labor dmeand.
$$ Z_{i} = L_{i,D} - L_i $$

If the wages are clearing the labor market, $Z_i$ should be zero for all the countries.


In [ ]:
Z = L_D - L_S
print(Z)

[ 0.14159292 -0.07079646 -0.07079646]


It does not. So the wages are not the equilibrium wages. We look for alternative wages. We write the current wages as $w^0_i$ and update them as follows:

$$ w_{new,i} = w_i * \left(1 + \psi * \frac{Z_i}{L_i}\right) $$

In [ ]:
psi = 0.1 # Parameter that governs the convergence speed.
w_new = w * (1 + psi * (Z / L_S) )
print(w_new)

[0.25353982 0.24882006 0.24882006]


Now we compare new wages and old wages. We confirm that the wages are higher in the countries where there are excess labor demands.

In [ ]:
print(Z)
print(w_new - w)

[ 0.14159292 -0.07079646 -0.07079646]
[ 0.00353982 -0.00117994 -0.00117994]


Equilbrium wages requires normalization. We normalize the wages so that the world GDP is one.

In [ ]:
# np.sum sums all the elements in the vector.
wgdp = np.sum(w_new * L_S)
print(wgdp)
# Divide the wages with the world GDP so that the world GDP will be one.
w_new = w_new / wgdp
print (np.sum(w_new * L_S))


1.0
1.0


# Function to calculate excess labor demand

To summarize the flow up to now:
1. Set the parameters
2. Calculate the import share and trade volume based on the assumed wages
3. Use the trade volume to calculate the excess demand for labor and update the tentatively placed wages.

We will summarize steps 2 to 3 into a **function**.


In [ ]:
def updatewage(w, theta, sigma, N, L_S, T, d, psi):
    """
    Updates wages based on parameters and variables.
    This function calculates new wage values based on input parameters and variables.

    Parameters:
    w (np.ndarray): Array of current wage rates for different countries.
    theta (float): Trade elasticity.
    sigma (float): Elasticity of substitution between goods.
    N (int): Number of countries.
    L_S (np.ndarray): Labor supply for each country.
    T (np.ndarray): Technology level for each country.
    d (np.ndarray): Trade costs between countries.
    psi (float): Wage adjustment parameter.

    Returns:
    w_new (np.ndarray): Updated wages for countries.
    Z (np.ndarray): Excess labor demand for each country.
    P (np.ndarray): Price index.
    X (np.ndarray): Trade flows between countries.
    """

    Xn = w * L_S

    ## Calculation of import share
    pi = np.zeros((N, N))  # Import share (the share of country i's goods in the total absorption of country n)
    pi_num = np.zeros((N, N))  # Numerator part of import share
    Phi = np.zeros((N))  # Denominator part of import share

    for OR, DE in np.ndindex((N, N)):
        pi_num[OR, DE] = T[OR] * (w[OR] * d[OR, DE]) ** (-theta)
        Phi[DE] += pi_num[OR, DE]

    for OR, DE in np.ndindex((N, N)):
        pi[OR, DE] = pi_num[OR, DE] / Phi[DE]

    # Calculation of price index
    P = gamma((theta + sigma - 1) / theta) ** (1 / (1 - sigma)) * Phi ** (-1 / theta)

    # Calculation of import value
    X = np.zeros((N, N))
    for OR, DE in np.ndindex((N, N)):
        X[OR, DE] = pi[OR, DE] * Xn[DE]

    # Calculation of excess labor demand
    L_D = np.zeros((N))
    for OR, DE in np.ndindex((N, N)):
        L_D[OR] += X[OR, DE] / w[OR]

    Z = L_D - L_S

    # Update and normalize wages
    w_new = w * (1 + psi * (Z / L_S))
    wgdp = np.sum(w_new * L_S)
    w_new = w_new / wgdp

    return w_new, Z, P, X

Confirm if the result from the fuction coincides with te result without the function.

In [ ]:
w_newfunc,_,_,_ = updatewage(w,theta=theta,sigma=sigma,N=N,L_S=L_S,T=T,d=d,psi=psi)
print(w_newfunc)
print(w_new)

[0.25353982 0.24882006 0.24882006]
[0.25353982 0.24882006 0.24882006]


# Introducing While-loop

Here, we  write code using a while loop to keep updating wages until the excess demand becomes sufficiently small. We define the threshold 'tol' here. A while loop is a loop that performs repeated calculations until a certain condition is met (while). In this case:

1. Calculate the excess labor demand using the tentatively placed wages
2. Update wages based on the excess labor demand
3. If the excess labor demand is greater than 'tol', go back to step 1
4. If the excess labor demand is smaller than 'tol', calculate the equilibrium outcomes of the economy and output them

In [ ]:
tol = 0.0001
iter = 1

# Set the initial values for the excess labor demand function and wages
Z = np.ones((N))
w = np.ones((N))

# If the excess labor demand is greater than tol, return here
while max(np.abs(Z)) > tol:
    iter += 1
    w_old = np.copy(w)
    w, Z, P, X = updatewage(w, theta=4, sigma=3, N=3, L_S=L_S, T=T, d=d, psi=0.1)

    # Display the number of iterations, excess labor demand, and wages every 10 iterations
    if iter % 10 == 0:
        print(iter)
        print(Z)
        print(w)

# When convergence is achieved, display the number of iterations required, the excess demand, and the wages
if max(np.abs(Z)) < tol:
    print("DONE!")
    print(iter)
    print(Z)
    print(w)

10
[ 0.00559218 -0.00294981 -0.00294981]
[0.26031388 0.24656204 0.24656204]
20
[ 1.13421104e-04 -5.99655974e-05 -5.99655974e-05]
[0.26061275 0.24646242 0.24646242]
DONE!
21
[ 7.68427249e-05 -4.06272769e-05 -4.06272769e-05]
[0.26061475 0.24646175 0.24646175]


Calculate the welfare:
$$ U_n = \frac{w_n}{P_n}. $$

In [ ]:
U = w / P
print(U)

[1.04077455 1.0157775  1.0157775 ]
